In [207]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import re
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import ensemble

In [219]:
#raw
injdata=pd.read_csv(r'C:\Users\jisaa\Desktop\New folder\injuries.csv')
injdata=injdata.drop(['Acquired'],axis=1)

#longterm injury
notes=injdata.Notes.values
LTI=[1 if 'out for season' in x else 0 for x in notes]
MTI=[1 if 'out indefinitely' in x else 0 for x in notes]
STI=[1 if 'DNP' in x or 'DTD' in x else 0 for x in notes]
injdata['LTI']=LTI
injdata['MTI']=MTI
injdata['STI']=STI
injdata=injdata[(injdata['MTI'] == 1) |(injdata['LTI'] == 1) | (injdata['STI'] == 1)]
#season
date=injdata.Date.values
seasonlist=list()
seasonendlist=list()

s11=datetime.strptime('April 16, 2011','%B %d, %Y')
s12=datetime.strptime('April 28, 2012','%B %d, %Y')
s13=datetime.strptime('April 20, 2013','%B %d, %Y')
s14=datetime.strptime('April 19, 2014','%B %d, %Y')
s15=datetime.strptime('April 18, 2015','%B %d, %Y')
s16=datetime.strptime('April 16, 2016','%B %d, %Y')
s17=datetime.strptime('April 15, 2017','%B %d, %Y')

for i in date:
    i=datetime.strptime(i,'%Y-%m-%d')
    if i<s11:
        season=2011
        season_end=s11
    elif i<s12:
        season=2012
        season_end=s12
    elif i<s13:
        season=2013
        season_end=s13
    elif i<s14:
        season=2014
        season_end=s14
    elif i<s15:
        season=2015
        season_end=s15
    elif i<s16:
        season=2016
        season_end=s16
    elif i<s17:
        season=2017
        season_end=s17
    else:
        season=2018
        season_end=datetime.strptime('April 14, 2018','%B %d, %Y')
    seasonlist.append(season)
    seasonendlist.append(season_end)
    
injdata['season']=seasonlist
injdata['season_end']=seasonendlist
injdata['season']=pd.to_datetime(injdata['season'],format='%Y')

rawname=injdata['Relinquised'].values
fname=list()
lname=list()
keyname=list()
for i in rawname:
    i=re.sub(r' /.+','',str(i))
    last=re.findall(r'(?<=\s).+$',i)
    if last:
        last=last[0]
    else:
        last=i
    lname.append(last)
    i=re.sub(last,'',i)
    first=i.strip()
    fname.append(first)
    if first:
        key=first[0]+"."+last
    else:
        key=last
    keyname.append(key)

injdata['Date']=pd.to_datetime(injdata['Date'],format='%Y-%m-%d')
injdata['fname']=fname
injdata['lname']=lname
injdata['keyname']=keyname
injdata=injdata.drop(['Relinquised'],axis=1)

injdata=injdata.dropna()

In [209]:
gamesmissedtemp=(injdata['season_end']-injdata['Date']).dt.days//7*2
LTI=injdata['LTI'].values
MTI=injdata['MTI'].values
newlist=list()

for i,j,k in zip(LTI,MTI,gamesmissedtemp):
    if i==1:
        newlist.append(k)
    elif j==1:
        newlist.append(5)
    else:
        newlist.append(1)
injdata['gamesmissed']=newlist

injdata=injdata.groupby(['season','Team','fname','lname','keyname'])['gamesmissed'].sum()
injdata=injdata.reset_index()

injdata=injdata.dropna()

In [210]:
teamdict=[{
'ATL':'Hawks',
'BKN':'Nets',
'BOS':'Celtics',
'CHA':'Hornets',
'CHI':'Bulls',
'CLE':'Cavaliers',
'DAL':'Mavericks',
'DEN':'Nuggets',
'DET':'Pistons',
'GSW':'Warriors',
'HOU':'Rockets',
'IND':'Pacers',
'LAC':'Clippers',
'LAL':'Lakers',
'MEM':'Grizzlies',
'MIA':'Heat',
'MIL':'Bucks',
'MIN':'Timberwolves',
'NOP':'Pelicans',
'NYK':'Knicks',
'OKC':'Thunder',
'ORL':'Magic',
'PHI':'76ers',
'PHX':'Suns',
'POR':'Blazers',
'SAC':'Kings',
'SAS':'Spurs',
'TOR':'Raptors',
'UTA':'Jazz',
'WAS':'Wizards'}]

team=pd.DataFrame.from_dict(teamdict)
team=team.transpose()
team.columns=["Team"]

In [211]:
statdata=pd.read_csv(r'C:\Users\jisaa\Desktop\New folder\Seasons_Stats.csv')
statdata=statdata.dropna(subset=['Year'])
                               
statdata=statdata.drop(statdata[statdata.Year<2011.0].index)
statdata['Year'] = pd.to_numeric(statdata["Year"],downcast='integer')
statdata['Year']=pd.to_datetime(statdata['Year'],format='%Y')
statdata.head(20)

rawname=statdata['Player'].values
keyname=list()
for i in rawname:
    last=re.findall(r'(?<=\s).+$',i)
    if last:
        last=last[0]
    else:
        last=i
    i=re.sub(last,'',i)
    first=i.strip()

    if first:
        key=first[0]+"."+last
    else:
        key=last
    keyname.append(key)
statdata['keyname']=keyname

statdata=pd.merge(statdata,team,how='inner',left_on=['Tm'],right_index=True)
statdata=statdata.drop(['blank2','blanl'],axis=1)

In [212]:
# statdataCOPY=statdata.drop(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm','keyname', 'Team'],axis=1)
# corr = statdataCOPY.corr().abs()
# fig, ax = plt.subplots(figsize=(45, 45))
# colormap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
# plt.xticks(range(len(corr.columns)), corr.columns);
# plt.yticks(range(len(corr.columns)), corr.columns)
# plt.show()

In [213]:
joindata=pd.merge(injdata,statdata, how='inner', left_on=['keyname','season','Team'],right_on=['keyname','Year','Team'])
joindata=joindata.drop(['Unnamed: 0','Year','Player','Tm'],axis=1)
joindata=joindata.dropna()

In [216]:
y=joindata['gamesmissed']
xnum=joindata.drop(['gamesmissed','season', 'Team', 'fname', 'lname', 'keyname', 'Pos'],axis=1)
xdum=joindata[['Pos']]

Yscaled=preprocessing.scale(Y)
xscaled=preprocessing.scale(xnum)
xdum=pd.get_dummies(xdum)

pca = PCA(n_components=15)
xpca = pca.fit_transform(xscaled)

xpca=pd.DataFrame(xpca)
xdum=pd.DataFrame(xdum)

xpca=xpca.reset_index()
xdum=xdum.reset_index()

x=xdum.join(xpca, lsuffix='index', rsuffix='index')
x=x.drop(['indexindex'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.50)

In [217]:
rf=ensemble.RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

y_pred= rf.predict(X_test)
errors = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'games.')

Mean Absolute Error: 8.55 degrees.
